In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import NearestNeighbors
from scipy import stats as st
import matplotlib.pyplot as plt
from PIL import Image
import scipy

In [2]:
def convolve_test(kernel_r, kernel_g, kernel_b, x_pixels, y_pixels, image_data, color=True):
    # Read in data as different arrays
    selected_image = np.empty((x_pixels,y_pixels,3)) # all channels
    selected_image_bw = np.empty((x_pixels,y_pixels))# luminosity
    selected_image_r = np.empty((x_pixels,y_pixels)) # red channel
    selected_image_g = np.empty((x_pixels,y_pixels)) # blue channel
    selected_image_b = np.empty((x_pixels,y_pixels)) # green channel

    index = 0
    for i in range(x_pixels):
        for j in range(y_pixels):
            selected_image_r[i][j] = 255*image_data[index+0]
            selected_image_g[i][j] = 255*image_data[index+1]
            selected_image_b[i][j] = 255*image_data[index+2]
            selected_image[i][j][0] = 255*image_data[index+0]
            selected_image[i][j][1] = 255*image_data[index+1]
            selected_image[i][j][2] = 255*image_data[index+2]
            selected_image_bw[i][j] = 0.3*selected_image[i][j][0] + 0.59*selected_image[i][j][1] + 0.11*selected_image[i][j][2]
            index += 3

    # And print it
    image = Image.fromarray(np.array(selected_image, dtype=np.uint8), mode="RGB")
    image.save('original.png')
    image = Image.fromarray(np.array(selected_image_bw, dtype=np.uint8), mode="P")
    image.save('original_bw.png')

    convolved_image = np.empty((x_pixels,y_pixels,3)) # all channels
    convolved_image_bw = np.empty((x_pixels,y_pixels)) # black and white
    convolved_image_difference = np.empty((x_pixels,y_pixels)) # difference between bw and original
    convolved_image_r = scipy.signal.convolve2d(selected_image_r,kernel_r) # red channel
    convolved_image_g = scipy.signal.convolve2d(selected_image_g,kernel_g) # blue channel
    convolved_image_b = scipy.signal.convolve2d(selected_image_b,kernel_b) # green channel
    bias_r = 1 - np.sum(kernel_r) 
    bias_g = 1 - np.sum(kernel_g) # bias terms if the kernel doesn't add up to 1
    bias_b = 1 - np.sum(kernel_b)
        
    for i in range(x_pixels):
        for j in range(y_pixels):
            convolved_image[i][j][0] = convolved_image_r[i][j]+bias_r
            convolved_image[i][j][1] = convolved_image_g[i][j]+bias_g
            convolved_image[i][j][2] = convolved_image_b[i][j]+bias_b
            convolved_image_bw[i][j] = 0.3*convolved_image[i][j][0] + 0.59*convolved_image[i][j][1] + 0.11*convolved_image[i][j][2]
    
    image = Image.fromarray(np.array(convolved_image, dtype=np.uint8), mode="RGB")
    image.save('convolved.png')
    image = Image.fromarray(np.array(convolved_image_bw, dtype=np.uint8), mode="P")
    image.save('convolved_bw.png')
    convolved_image_difference = np.abs(np.add(convolved_image_bw,-1*selected_image_bw))
    image = Image.fromarray(np.array(convolved_image_difference, dtype=np.uint8), mode="P")
    image.save('convolved_difference.png')

In [30]:
def h(x): # ReLU
    y = np.maximum(0,x)
    return y

def convolve_layer(kernel_r, kernel_g, kernel_b, x_pixels, y_pixels, image_data, color=True):
    # Read in data in different channels
    selected_image_r = np.empty((x_pixels,y_pixels)) # red channel
    selected_image_g = np.empty((x_pixels,y_pixels)) # blue channel
    selected_image_b = np.empty((x_pixels,y_pixels)) # green channel

    index = 0
    for i in range(x_pixels):
        for j in range(y_pixels):
            selected_image_r[i][j] = 255*image_data[index+0]
            selected_image_g[i][j] = 255*image_data[index+1]
            selected_image_b[i][j] = 255*image_data[index+2]
            index += 3

    # Convolution, only valid entries - this reduces the dimensionality
    # following https://saturncloud.io/blog/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way/
    convolved_image_r = scipy.signal.convolve2d(selected_image_r,kernel_r,mode='valid') # red channel
    convolved_image_g = scipy.signal.convolve2d(selected_image_g,kernel_g,mode='valid') # blue channel
    convolved_image_b = scipy.signal.convolve2d(selected_image_b,kernel_b,mode='valid') # green channel
    x_pixels_convolved = convolved_image_r.shape[0]
    y_pixels_convolved = convolved_image_r.shape[1]
    convolved_image = np.empty((x_pixels_convolved,y_pixels_convolved))
        
    for i in range(x_pixels_convolved):
        for j in range(y_pixels_convolved):
            convolved_image[i][j] = h(0.3*convolved_image_r[i][j] + 0.59*convolved_image_g[i][j] + 0.11*convolved_image_b[i][j])

    image = Image.fromarray(np.array(convolved_image, dtype=np.uint8), mode="P")
    image.save('convolved.png')
    return np.ravel(convolved_image)

In [31]:
from sklearn.datasets import fetch_lfw_people
color = True
people = fetch_lfw_people(min_faces_per_person=8, resize=0.5, color=color)

x = people.data 
y = people.target

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75)

print("Total data points, x pixels, y pixels, color channels")
print(people.images.shape)
print(x_train.shape)
print(x_test.shape)

Total data points, x pixels, y pixels, color channels
(4503, 62, 47, 3)
(3377, 8742)
(1126, 8742)


In [ ]:
# Let's do the convolutions
x_pixels = people.images.shape[1]
y_pixels = people.images.shape[2]
#kernel = np.array([[-1, -1, -1, -1, -1],
#                   [-1, -1, -1, -1, -1],
#                   [-1, -1, 25, -1, -1],
#                   [-1, -1, -1, -1, -1],
#                   [-1, -1, -1, -1, -1],])
kernel = np.array([[-1, -1, -1],
                   [-1,  8, -1],
                   [-1, -1, -1]])

new_dimensionality = int((x_train.shape[1]/3) - (kernel.shape[0]-1)*people.images.shape[1] - (kernel.shape[0]-1)*people.images.shape[2] + 4*((kernel.shape[0]-1)/2)**2)

x_train_new = np.empty((x_train.shape[0],new_dimensionality))
x_test_new = np.empty((x_test.shape[0],new_dimensionality))
print("Starting testing data")
for i in range(len(x_test)):
    if i % 250 ==0:
        print(i)
    x_test_new[i] = convolve_layer(kernel, kernel, kernel, x_pixels, y_pixels, x_train[i])
print("Starting training data")
for i in range(len(x_train)):
    if i % 250 ==0:
        print(i)
    x_train_new[i] = convolve_layer(kernel, kernel, kernel, x_pixels, y_pixels, x_train[i])

# we should do a max pool

print("After convolutions:")
print(x_train_new.shape)
print(x_test_new.shape)

Starting testing data
0
250
500
750
1000
Starting training data
0
250
500
750


In [39]:
# kNN
k=100
model = NearestNeighbors(n_neighbors=k)
model.fit(x_train_new,y_train)
test_neighbors = model.kneighbors_graph(x_test_new)
print("Picking " + str(k) + " nearest neighbors, a total of " + str(x_train_new.shape[0]) + " training points with " + str(x_train_new.shape[1]) + " dimensions")

Picking 100 nearest neighbors, a total of 3377 training points with 2700 dimensions


In [40]:
# test_nearest_neighbors looks through test_neighbors and gets the classifications of the k nearest ones
test_nearest_neighbors = np.empty((y_test.shape[0],k))
# y_pred gets the mode of the nearest neighbors
y_pred = np.empty((y_test.shape[0],1))

print("Total number of test points: " + str(x_test.shape[0]))
for i in range(x_test_new.shape[0]):
    if i%50 == 0:
        print("i=" + str(i))
    test_nearest_neighbors[i] = y_train[np.nonzero(test_neighbors.toarray()[i])]
    y_pred[i] = int(st.mode(test_nearest_neighbors[i], keepdims=True).mode)

print("Done!")

Total number of test points: 1126
i=0
i=50
i=100
i=150
i=200
i=250
i=300
i=350
i=400
i=450
i=500
i=550
i=600
i=650
i=700
i=750
i=800
i=850
i=900
i=950
i=1000
i=1050
i=1100
Done!


In [41]:
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)
print("Accuracy: " + str(np.round(100*conf_matrix.trace()/conf_matrix.sum(),2)) + "%")

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Accuracy: 12.7%
